In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [4]:
def create_dataset(data, time_step):
    dataX = []
    for i in range(data.shape[0] - time_step + 1):
        dataX.append(data[i:i + time_step])
    return np.array(dataX)

In [5]:
def read_raw_data(vtype, depth, time_step):
    train_argo = np.load('/media/aita-ocean/data/WILL/1103Argo/mutichannel/data/'+vtype+'_0_ano.npy')#读取数据，指定维度标签
    label_argo = np.load('/media/aita-ocean/data/WILL/1103Argo/mutichannel/data/'+vtype+'_'+str(depth)+'_ano.npy')#206,60,80
    width = train_argo.shape[2] #对应经度
    lenth = train_argo.shape[1] #对应纬度
    X = create_dataset(train_argo, time_step)
    X = X.reshape(X.shape[0],time_step,lenth,width,1)
    Y = label_argo[time_step-1 : label_argo.shape[0]] 
    Y =Y.reshape(Y.shape[0],lenth,width,1)
    X = X.transpose(0,1,4,2,3)
    Y = Y.transpose(0,3,1,2)
    return X, Y

In [6]:
def scaler(data):
    #normalise [0,1]
    data_max = np.nanmax(data)
    data_min = np.nanmin(data)
    data_scale = data_max - data_min
    data_std = (data - data_min) / data_scale
    # data_std = data_std * (2)  -1
    data_std [np.isnan(data_std)] = 0
    return data_std,data_min,data_scale
'''
将nan替换为0，也就是说nan的异常为0，即nan的值为均值
'''

def unscaler(data, data_min, data_scale):
    data_inv = (data * data_scale) + data_min
    return data_inv

In [7]:
train_sssa, _ = read_raw_data('salt', 0, 3)
train_ssha,_ = read_raw_data('sla',0,3)
train_sswu,_ = read_raw_data('uwnd',0,3)
train_sswv,_ = read_raw_data('vwnd',0,3)
train_argo, label_argo = read_raw_data('temp', 2, 3)# 要改

In [8]:
sta_train,_,_ = scaler(train_argo[:-12,:])
ssa_train,_,_  = scaler(train_sssa[:-12,:])
ssha_train,_,_ = scaler(train_ssha[:-12,:])
sswu_train,_,_ = scaler(train_sswu[:-12,:])
sswv_train,_,_ = scaler(train_sswv[:-12,:])
true_train,_,_ = scaler(label_argo[:-12,:])

In [9]:
print(sta_train.shape)
print(ssa_train.shape)
print(ssha_train.shape)
print(sswu_train.shape)

(166, 3, 1, 60, 80)
(166, 3, 1, 60, 80)
(166, 3, 1, 60, 80)
(166, 3, 1, 60, 80)


In [10]:
sta_test,_,_ = scaler(train_argo[-12:])
ssa_test,_,_  = scaler(train_sssa[-12:])
ssha_test,_,_ = scaler(train_ssha[-12:])
sswu_test,_,_ = scaler(train_sswu[-12:])
sswv_test,_,_ = scaler(train_sswv[-12:])

In [11]:
sta_train = np.concatenate((sta_train,ssa_train,ssha_train,sswu_train,sswv_train),axis = 2 )
sta_test = np.concatenate((sta_test,ssa_test,ssha_test,sswu_test,sswv_test),axis = 2)

In [12]:
true_test,test_min,test_scale = scaler(label_argo[-12:])

In [47]:
true_train,test_min1,test_scale1= scaler(label_argo[:-12,:])

In [48]:
true_train.shape

(166, 1, 60, 80)

In [13]:
X_train = sta_train
true_train = true_train


X_eval = sta_test
true_eval = true_test
X_test = sta_test
true_test = true_test

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pickle
import numpy as np
import math
from torch.utils.data import Dataset

In [15]:
class Configs:
    def __init__(self):
        pass


configs = Configs()

# trainer related
configs.vtype = 'temp'
# configs.depth = 11
# configs.time_step = 1
configs.n_cpu = 0
configs.device = torch.device('cuda:0')
configs.batch_size_test = 100
configs.batch_size = 32
#configs.lr = 0.001
configs.weight_decay = 0
configs.display_interval = 200
configs.num_epochs = 900
configs.early_stopping = True
configs.patience = 900
configs.gradient_clipping = False
configs.clipping_threshold = 1.

# lr warmup
configs.warmup = 3000

# data related
configs.input_dim = 1 # 4 #这里应该是5吧 但是写的1我总感觉是5
'''
人家这个1是对的这个模型就是要保证输入通道和输出通道得一样
默认为1
'''
configs.output_dim = 1

configs.input_length = 5
configs.output_length = 1

configs.input_gap = 1
configs.pred_shift = 32
configs.depth = [5,6,11,16,20,25,30,34,36,38,40,42,44,46,48,50,51,52,53,54,55,57]
configs.depthindex = [30,50, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900]

# model
configs.d_model = 256
configs.patch_size = (5,5)
configs.emb_spatial_size = 12*16
configs.nheads = 4
configs.dim_feedforward = 512
configs.dropout = 0.3
configs.num_encoder_layers = 4
configs.num_decoder_layers = 4

configs.ssr_decay_rate = 3.e-5


# plot
configs.plot_dpi = 600


In [16]:
class covlstmformer(nn.Module):
    def __init__(self, configs):
        super().__init__()
        self.configs = configs
        self.d_model = 25
        self.device = configs.device
        self.cov1 = Cov(5, 8,3, 5)
        self.cov2 = Cov(5, 8,3, 5)
        self.encode1 = EncoderLayer(self.d_model, 1, configs.dim_feedforward, configs.dropout)
        self.encode2 = EncoderLayer(self.d_model, 1, configs.dim_feedforward, configs.dropout)
        self.cov_last  = Cov_last(5, 8,3, 1)
    def forward(self,x):
        resdual1 = self.cov1(x)
        resdual1 = unfold_StackOverChannel(resdual1, (5, 5))
        x = resdual1
        x = resdual1 + self.encode1(x)
        x = fold_tensor(x, (60, 80), (5, 5))
        resdual2 = x + self.cov2(x) # xiu gai 的地方在这
        resdual2 = unfold_StackOverChannel(resdual2, (5, 5))
        x = resdual2
        x = resdual2 + self.encode2(x)
        x = fold_tensor(x, (60, 80), (5, 5))
        x = self.cov_last(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, nheads, dim_feedforward, dropout):
        super().__init__()
        self.time_attn = MultiHeadedAttention(d_model, nheads, TimeAttention, dropout)
        self.space_attn = MultiHeadedAttention(d_model, nheads, SpaceAttention, dropout)
        '''
        self.net = nn.Sequential(
                  nn.Linear(256, 25),
                  nn.ReLU(),
                  nn.Linear(25, 256),
                   nn.ReLU(),
                   nn.Linear(256, 512),
                   nn.ReLU(),
                   nn.Linear(512,256)
                  )
        '''
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, d_model)
            )
    '''
    def divided_space_time_attn(self, query, key, value, mask):
        """
        Apply space and time attention sequentially
        Args:
            query (N, S, T, D)
            key (N, S, T, D)
            value (N, S, T, D)
        Returns:
            (N, S, T, D)
        """
        m = self.time_attn(query, key, value, mask)
        return self.space_attn(m, m, m, mask)
    '''
    def forward(self, x, mask=None):
        # x = self.sublayer[0](x, lambda x: self.divided_space_time_attn(x, x, x, mask))
        # x = x + self.net(x)
        # return self.sublayer[1](x, self.feed_forward)
        x = x + self.time_attn(x, x, x, mask)
        x = x+ self.space_attn(x, x, x,mask)
        x = x + self.feed_forward(x)
        return x
class ConvLSTMCell(nn.Module):
 
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """
 
        super(ConvLSTMCell, self).__init__()
 
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
 
        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2 # 保证在传递过程中 （h,w）不变
        self.bias = bias
 
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim, # i门，f门，o门，g门放在一起计算，然后在split开
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)
 
    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state # 每个timestamp包含两个状态张量：h和c
 
        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis # 把输入张量与h状态张量沿通道维度串联
 
        combined_conv = self.conv(combined) # i门，f门，o门，g门放在一起计算，然后在split开
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)
 
        c_next = f * c_cur + i * g  # c状态张量更新
        h_next = o * torch.tanh(c_next) # h状态张量更新
 
        return h_next, c_next # 输出当前timestamp的两个状态张量
 
    def init_hidden(self, batch_size, image_size):
        """
        初始状态张量初始化.第一个timestamp的状态张量0初始化
        :param batch_size:
        :param image_size:
        :return:
        """
        height, width = image_size
        init_h = torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
        init_c = torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
        return (init_h,init_c)
 
 
class ConvLSTM(nn.Module):
 
    """
    Parameters:参数介绍
        input_dim: Number of channels in input# 输入张量的通道数
        hidden_dim: Number of hidden channels # h,c两个状态张量的通道数，可以是一个列表
        kernel_size: Size of kernel in convolutions # 卷积核的尺寸，默认所有层的卷积核尺寸都是一样的,也可以设定不通lstm层的卷积核尺寸不同
        num_layers: Number of LSTM layers stacked on each other # 卷积层的层数，需要与len(hidden_dim)相等
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers # 是否返回所有lstm层的h状态
        Note: Will do same padding. # 相同的卷积核尺寸，相同的padding尺寸
    Input:输入介绍
        A tensor of size [B, T, C, H, W] or [T, B, C, H, W]# 需要是5维的
    Output:输出介绍
        返回的是两个列表：layer_output_list，last_state_list
        列表0：layer_output_list--单层列表，每个元素表示一层LSTM层的输出h状态,每个元素的size=[B,T,hidden_dim,H,W]
        列表1：last_state_list--双层列表，每个元素是一个二元列表[h,c],表示每一层的最后一个timestamp的输出状态[h,c],h.size=c.size = [B,hidden_dim,H,W]
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:使用示例
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """
 
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()
 
        self._check_kernel_size_consistency(kernel_size)
 
        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers) # 转为列表
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers) # 转为列表
        if not len(kernel_size) == len(hidden_dim) == num_layers: # 判断一致性
            raise ValueError('Inconsistent list length.')
 
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers
 
        cell_list = []
        for i in range(0, self.num_layers): # 多层LSTM设置
            # 当前LSTM层的输入维度
            # if i==0:
            #     cur_input_dim = self.input_dim
            # else:
            #     cur_input_dim = self.hidden_dim[i - 1]
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1] # 与上等价
            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))
 
        self.cell_list = nn.ModuleList(cell_list) # 把定义的多个LSTM层串联成网络模型
 
    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: 5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)
 
        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            b, _, _, h, w = input_tensor.size()  # 自动获取 b,h,w信息
            hidden_state = self._init_hidden(batch_size=b,image_size=(h, w))
 
        layer_output_list = []
        last_state_list = []
 
        seq_len = input_tensor.size(1) # 根据输入张量获取lstm的长度
        cur_layer_input = input_tensor
 
        for layer_idx in range(self.num_layers): # 逐层计算
 
            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len): # 逐个stamp计算
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],cur_state=[h, c])
                output_inner.append(h) # 第 layer_idx 层的第t个stamp的输出状态
 
            layer_output = torch.stack(output_inner, dim=1) # 第 layer_idx 层的第所有stamp的输出状态串联
            cur_layer_input = layer_output # 准备第layer_idx+1层的输入张量
 
            layer_output_list.append(layer_output) # 当前层的所有timestamp的h状态的串联
            last_state_list.append([h, c]) # 当前层的最后一个stamp的输出状态的[h,c]
 
        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]
 
        return layer_output_list, last_state_list
 
    def _init_hidden(self, batch_size, image_size):
        """
        所有lstm层的第一个timestamp的输入状态0初始化
        :param batch_size:
        :param image_size:
        :return:
        """
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states
 
    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        """
        检测输入的kernel_size是否符合要求，要求kernel_size的格式是list或tuple
        :param kernel_size:
        :return:
        """
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')
 
    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        """
        扩展到多层lstm情况
        :param param:
        :param num_layers:
        :return:
        """
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

class Cov(nn.Module):
     def __init__(self, intput_dim, hidden_dim, bn_dim, output_dim):# bn_dim是时间步
          super().__init__()
          self.cov1 = ConvLSTM(input_dim = intput_dim,
                     hidden_dim = hidden_dim,
                     kernel_size=(3, 3),
                     num_layers=1,
                     batch_first=True,
                     bias = True,
                     return_all_layers = True)
          self.bn1 = nn.BatchNorm3d(bn_dim)
          self.cov2 = ConvLSTM(input_dim = hidden_dim,
                     hidden_dim = hidden_dim,
                     kernel_size=(3, 3),
                     num_layers=1,
                     batch_first=True,
                     bias = True,
                     return_all_layers = True)
          self.bn2 = nn.BatchNorm3d(bn_dim)
          self.cov3 = ConvLSTM(input_dim = hidden_dim,
                     hidden_dim = output_dim,
                     kernel_size=(3, 3),
                     num_layers=1,
                     batch_first=True,
                     bias = True,
                     return_all_layers = True)
     def forward(self, x):
          x,_ = self.cov1(x) # 因为上面的Covlstm返回两个值所以先用_接住第二个用不到的值
          x = self.bn1(x[0])
          x,_ = self.cov2(x)
          x = self.bn2(x[0])
          x,_ = self.cov3(x)
          x = x[0]
          return x
     
class Cov_last(nn.Module):
     def __init__(self, intput_dim, hidden_dim, bn_dim, output_dim):# bn_dim是时间步
          super().__init__()
          self.cov1 = ConvLSTM(input_dim = intput_dim,
                     hidden_dim = hidden_dim,
                     kernel_size=(3, 3),
                     num_layers=1,
                     batch_first=True,
                     bias = True,
                     return_all_layers = True)
          self.bn1 = nn.BatchNorm3d(bn_dim)
          self.cov2 = ConvLSTM(input_dim = hidden_dim,
                     hidden_dim = hidden_dim,
                     kernel_size=(3, 3),
                     num_layers=1,
                     batch_first=True,
                     bias = True,
                     return_all_layers = True)
          self.bn2 = nn.BatchNorm3d(bn_dim)
          self.cov3 = ConvLSTM(input_dim = hidden_dim,
                     hidden_dim = output_dim,
                     kernel_size=(3, 3),
                     num_layers=1,
                     batch_first=True,
                     bias = True,
                     return_all_layers = True)
     def forward(self, x):
          x,_ = self.cov1(x) # 因为上面的Covlstm返回两个值所以先用_接住第二个用不到的值
          x = self.bn1(x[0])
          x,_ = self.cov2(x)
          x = self.bn2(x[0])
          x,_ = self.cov3(x)
          x = x[0]
          return x[:, -1] 
     
def unfold_StackOverChannel(img, kernel_size):
    """
    divide the original image to patches, then stack the grids in each patch along the channels
    Args:
        img (N, *, C, H, W): the last two dimensions must be the spatial dimension
        kernel_size: tuple of length 2
    Returns:
        output (N, *, C*H_k*N_k, H_output, W_output)
    """
    T = img.size(1)
    n_dim = len(img.size())
    assert n_dim == 4 or n_dim == 5

    pt = img.unfold(-2, size=kernel_size[0], step=kernel_size[0])
    pt = pt.unfold(-2, size=kernel_size[1], step=kernel_size[1]).flatten(-2)  # (N, *, C, n0, n1, k0*k1)
    if n_dim == 4:  # (N, C, H, W)
        pt = pt.permute(0, 1, 4, 2, 3).flatten(1, 2)
    elif n_dim == 5:  # (N, T, C, H, W)
        pt = pt.permute(0, 1, 2, 5, 3, 4).flatten(2, 3)
    assert pt.size(-3) == img.size(-3) * kernel_size[0] * kernel_size[1]
    pt = pt.reshape(pt.size(0), T, 25, -1).permute(0, 3, 1, 2)
    return pt     
def fold_tensor(tensor, output_size, kernel_size):
    """
    reconstruct the image from its non-overlapping patches
    Args:
        input tensor of size (N, *, C*k_h*k_w, n_h, n_w)
        output_size of size(H, W), the size of the original image to be reconstructed
        kernel_size: (k_h, k_w)
        stride is usually equal to kernel_size for non-overlapping sliding window
    Returns:
        (N, *, C, H=n_h*k_h, W=n_w*k_w)
    """
    tensor = tensor.reshape(-1,192,3,25)
    T = tensor.size(2)
    tensor = tensor.permute(0, 2, 3, 1)  # (N, T, C_, S)
    tensor = tensor.reshape(tensor.size(0), T, 25,
                                12, 16)
    tensor = tensor.float()
    n_dim = len(tensor.size())
    assert n_dim == 4 or n_dim == 5
    f = tensor.flatten(0, 1) if n_dim == 5 else tensor
    folded = F.fold(f.flatten(-2), output_size=output_size, kernel_size=kernel_size, stride=kernel_size)
    if n_dim == 5:
        folded = folded.reshape(tensor.size(0), tensor.size(1), *folded.size()[1:])
    return folded.reshape(-1,T,5,60,80)


def TimeAttention(query, key, value, mask=None, dropout=None):
    """
    attention over the time axis
    Args:
        query, key, value: linearly-transformed query, key, value (N, h, S, T, D)
        mask: of size (T (query), T (key)) specifying locations (which key) the query can and cannot attend to
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / np.sqrt(d_k)  # (N, h, S, T, T)
    if mask is not None:
        assert mask.dtype == torch.bool
        assert len(mask.size()) == 2
        scores = scores.masked_fill(mask[None, None, None], float("-inf"))
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value)  # (N, h, S, T, D)


def SpaceAttention(query, key, value, mask=None, dropout=None):
    """
    attention over the two space axes
    Args:
        query, key, value: linearly-transformed query, key, value (N, h, S, T, D)
        mask: None (space attention does not need mask), this argument is intentionally set for consistency
    """
    d_k = query.size(-1)
    query = query.transpose(2, 3)  # (N, h, T, S, D)
    key = key.transpose(2, 3)  # (N, h, T, S, D)
    value = value.transpose(2, 3)  # (N, h, T, S, D)
    scores = torch.matmul(query, key.transpose(-2, -1)) / np.sqrt(d_k)  # (N, h, T, S, S)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value).transpose(2, 3)  # (N, h, S, T_q, D)
class MultiHeadedAttention(nn.Module):
    def __init__(self, d_model, nheads, attn, dropout):
        super().__init__()
        assert d_model % nheads == 0
        self.d_k = d_model // nheads
        self.nheads = nheads
        self.linears = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(4)])
        self.dropout = nn.Dropout(p=dropout)
        self.attn = attn

    def forward(self, query, key, value, mask=None):
        """
        Transform the query, key, value into different heads, then apply the attention in parallel
        Args:
            query, key, value: size (N, S, T, D)
        Returns:
            (N, S, T, D)
        """
        nbatches = query.size(0)
        nspace = query.size(1)
        ntime = query.size(2)
        # (N, h, S, T, d_k)
        query, key, value = \
            [l(x).view(x.size(0), x.size(1), x.size(2), self.nheads, self.d_k).permute(0, 3, 1, 2, 4)
             for l, x in zip(self.linears, (query, key, value))]

        # (N, h, S, T, d_k)
        x = self.attn(query, key, value, mask=mask, dropout=self.dropout)

        # (N, S, T, D)
        x = x.permute(0, 2, 3, 1, 4).contiguous() \
             .view(nbatches, nspace, ntime, self.nheads * self.d_k)
        return self.linears[-1](x)
    

class Attention(nn.Module):
    def __init__(self, dropout,attn):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(192, 192) for _ in range(3)])
        self.attn = attn
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, query, key, value, mask=None):
        query, key, value = \
        [l(x)
        for l, x in zip(self.linears, (query, key, value))]
        x = self.attn(query, key, value, mask=mask, dropout=self.dropout)
        return x


In [17]:
class NoamOpt:
    """
    learning rate warmup and decay
    """

    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0

    def step(self):
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()

    def rate(self, step=None):
        if step is None:
            step = self._step
        return self.factor * \
               (self.model_size ** (-0.5) * min(step ** (-0.5), step * self.warmup ** (-1.5)))


class Trainer:
    def __init__(self, configs):
        self.configs = configs
        self.device = configs.device
        torch.manual_seed(5)
        self.network  = covlstmformer(configs).to(configs.device)
        adam = torch.optim.Adam(self.network.parameters(), lr=0, weight_decay=configs.weight_decay)
        factor = math.sqrt(configs.d_model * configs.warmup) * 0.0014
        self.opt = NoamOpt(configs.d_model, factor, warmup=configs.warmup, optimizer=adam)


    def loss_sst(self, y_pred, y_true):
        # y_pred/y_true (N, 26, 24, 48)
        rmse = torch.mean((y_pred - y_true) ** 2, dim=[2, 3])
        rmse = torch.sum(rmse.sqrt().mean(dim=0))
        return rmse



    def train_once(self, input_sst, sst_true, ssr_ratio):
        sst_pred = self.network(input_sst.float().to(self.device))
        self.opt.optimizer.zero_grad()
        loss_sst = self.loss_sst(sst_pred, sst_true.float().to(self.device))
        # loss_nino = self.loss_nino(nino_pred, nino_true.float().to(self.device))
        loss_sst.backward()
        if configs.gradient_clipping:
            nn.utils.clip_grad_norm_(self.network.parameters(), configs.clipping_threshold)
        self.opt.step()
        return loss_sst.item()

    def test(self, dataloader_test):
        # nino_pred = []
        sst_pred = []
        with torch.no_grad():
            for input_sst, sst_true, in dataloader_test:
                sst = self.network(input_sst.float().to(self.device))
                # nino_pred.append(nino)
                sst_pred.append(sst)

        return torch.cat(sst_pred, dim=0)

    def infer(self, dataset, dataloader):
        self.network.eval()
        with torch.no_grad():
            sst_pred = self.test(dataloader)
            # nino_true = torch.from_numpy(dataset.target_nino).float().to(self.device)
            sst_true = torch.from_numpy(dataset.target_sst).float().to(self.device)
            # sc = self.score(nino_pred, nino_true)
#             print(sst_pred.shape)
#             print(sst_true.shape)
            loss_sst = self.loss_sst(sst_pred, sst_true).item()
            # loss_nino = self.loss_nino(nino_pred, nino_true).item()
        return loss_sst

    def infer_test(self, dataset, dataloader):
        self.network.eval()
        with torch.no_grad():
            sst_pred = self.test(dataloader)
            # nino_true = torch.from_numpy(dataset.target_nino).float().to(self.device)
            sst_true = torch.from_numpy(dataset.target_sst).float().to(self.device)
            # sc = self.score(nino_pred, nino_true)
            loss_sst = self.loss_sst(sst_pred, sst_true).item()
            # loss_nino = self.loss_nino(nino_pred, nino_true).item()
        return loss_sst, sst_pred, sst_true

    def train(self, dataset_train, dataset_eval, chk_path):
        torch.manual_seed(0)
        print('loading train dataloader')
        dataloader_train = DataLoader(dataset_train, batch_size=self.configs.batch_size, shuffle=True)
        print('loading eval dataloader')
        dataloader_eval = DataLoader(dataset_eval, batch_size=self.configs.batch_size_test, shuffle=False)

        count = 0
        best = math.inf
        ssr_ratio = 1
        for i in range(self.configs.num_epochs):
            print('\nepoch: {0}'.format(i + 1))
            # train
            self.network.train()
            for j, (input_sst, sst_true) in enumerate(dataloader_train):
                if ssr_ratio > 0:
                    ssr_ratio = max(ssr_ratio - self.configs.ssr_decay_rate, 0)
                loss_sst = self.train_once(input_sst, sst_true, ssr_ratio)  # y_pred for one batch

                if j % self.configs.display_interval == 0:

                    print('batch training loss: {:.5f}, ssr: {:.5f}, lr: {:.5f}'.format(loss_sst, ssr_ratio, self.opt.rate()))

                # increase the number of evaluations in order not to miss the optimal point
                # which is feasible because of the less training time of timesformer
                if (i + 1 >= 9) and (j + 1) % 300 == 0:
                    loss_sst_eval = self.infer(dataset=dataset_eval, dataloader=dataloader_eval)
                    print('epoch eval loss: sc: {:.4f}'.format(loss_sst_eval))
                    if loss_sst_eval < best:
                        self.save_model(chk_path)
                        best = loss_sst_eval
                        count = 0

            # evaluation
            loss_sst_eval = self.infer(dataset=dataset_eval, dataloader=dataloader_eval)
            print('epoch eval loss:\nsst: {:.2f}'.format(loss_sst_eval))
            if loss_sst_eval >= best:
                count += 1
                print('eval score is not improved for {} epoch'.format(count))
            else:
                count = 0
                print('eval score is improved from {:.5f} to {:.5f}, saving model'.format(best, loss_sst_eval))
                self.save_model(chk_path)
                best = loss_sst_eval

            if count == self.configs.patience:
                print('early stopping reached, best score is {:5f}'.format(best))
                break

    def save_configs(self, config_path):
        with open(config_path, 'wb') as path:
            pickle.dump(self.configs, path)

    def save_model(self, path):
        torch.save({'net': self.network.state_dict(),
                    'optimizer': self.opt.optimizer.state_dict()}, path)
        

In [18]:
class cmip_dataset(Dataset):
    def __init__(self, datax,datay):
        super().__init__()

        self.input_sst = datax
        self.target_sst = datay


    def GetDataShape(self):
        return {'sst input': self.input_sst.shape,
                'sst target': self.target_sst.shape}

    def __len__(self,):
        return self.input_sst.shape[0]

    def __getitem__(self, idx):
        return self.input_sst[idx], self.target_sst[idx]

In [19]:
dataset_train = cmip_dataset(X_train,true_train)
print(dataset_train.GetDataShape())

{'sst input': (166, 3, 5, 60, 80), 'sst target': (166, 1, 60, 80)}


In [20]:
dataset_eval = cmip_dataset(X_eval,true_eval)
print(dataset_eval.GetDataShape())

{'sst input': (12, 3, 5, 60, 80), 'sst target': (12, 1, 60, 80)}


In [21]:
trainer = Trainer(configs)
#trainer.save_configs('config_train.pkl')

In [22]:
trainer.train(dataset_train, dataset_eval, 'checkpoint.chk')

loading train dataloader
loading eval dataloader

epoch: 1
batch training loss: 0.50818, ssr: 0.99997, lr: 0.00000
epoch eval loss:
sst: 0.52
eval score is improved from inf to 0.52345, saving model

epoch: 2
batch training loss: 0.51119, ssr: 0.99979, lr: 0.00000
epoch eval loss:
sst: 0.52
eval score is improved from 0.52345 to 0.52304, saving model

epoch: 3
batch training loss: 0.49602, ssr: 0.99961, lr: 0.00001
epoch eval loss:
sst: 0.52
eval score is improved from 0.52304 to 0.52088, saving model

epoch: 4
batch training loss: 0.48438, ssr: 0.99943, lr: 0.00001
epoch eval loss:
sst: 0.52
eval score is improved from 0.52088 to 0.51543, saving model

epoch: 5
batch training loss: 0.47274, ssr: 0.99925, lr: 0.00001
epoch eval loss:
sst: 0.50
eval score is improved from 0.51543 to 0.50441, saving model

epoch: 6
batch training loss: 0.45646, ssr: 0.99907, lr: 0.00001
epoch eval loss:
sst: 0.49
eval score is improved from 0.50441 to 0.48551, saving model

epoch: 7
batch training loss: 

In [24]:
dataset_test = cmip_dataset(X_test, true_test)
dataloader_test = DataLoader(dataset_test, batch_size=configs.batch_size_test, shuffle=False)
print(dataset_test.GetDataShape())

{'sst input': (12, 3, 5, 60, 80), 'sst target': (12, 1, 60, 80)}


In [25]:
chk = torch.load('/media/aita-ocean/data/XGX_NEW/argo/cf/depth2/checkpoint.chk')


In [26]:
trainer.network.load_state_dict(chk['net'])

<All keys matched successfully>

In [40]:
print(sta_train.shape)
print(sta_test.shape)

(166, 3, 5, 60, 80)
(12, 3, 5, 60, 80)


In [41]:
xx = np.concatenate((sta_train, sta_test),0)
xx.shape

(178, 3, 5, 60, 80)

In [43]:
xx = torch.from_numpy(xx).float().to(configs.device)
xx

tensor([[[[[0.6819, 0.6906, 0.6900,  ..., 0.5225, 0.5414, 0.6116],
           [0.6718, 0.6797, 0.6828,  ..., 0.5408, 0.5661, 0.6369],
           [0.6618, 0.6628, 0.6664,  ..., 0.5736, 0.5951, 0.6399],
           ...,
           [0.1477, 0.1850, 0.2534,  ..., 0.3844, 0.3601, 0.3801],
           [0.1368, 0.1756, 0.2520,  ..., 0.4087, 0.3753, 0.3869],
           [0.1290, 0.1596, 0.2449,  ..., 0.4524, 0.4132, 0.4045]],

          [[0.6058, 0.6024, 0.6012,  ..., 0.4539, 0.4677, 0.5232],
           [0.5994, 0.5992, 0.5991,  ..., 0.4687, 0.4890, 0.5480],
           [0.5942, 0.5967, 0.5980,  ..., 0.4852, 0.5150, 0.5631],
           ...,
           [0.6190, 0.6237, 0.6340,  ..., 0.4901, 0.4816, 0.5155],
           [0.6178, 0.6209, 0.6377,  ..., 0.5117, 0.4980, 0.5283],
           [0.6200, 0.6188, 0.6439,  ..., 0.5536, 0.5336, 0.5505]],

          [[0.6307, 0.5170, 0.5376,  ..., 0.4969, 0.4141, 0.3436],
           [0.4849, 0.4463, 0.5330,  ..., 0.5214, 0.3832, 0.3959],
           [0.3131, 0.3577

In [44]:
depth2_ano = trainer.network(xx)

In [46]:
depth2_ano.shape

torch.Size([178, 1, 60, 80])

In [50]:
depth2_ano = depth2_ano.cpu().detach().numpy()

In [52]:
depth2_ano.shape

(178, 1, 60, 80)

In [29]:
test_pred = test_pred.cpu().numpy()
test_true = test_true.cpu().numpy()

In [30]:
test_pred = unscaler(np.array(test_pred),test_min,test_scale)
test_true = unscaler(np.array(test_true),test_min,test_scale)

In [31]:
test_pred = test_pred.transpose(0,2,3,1)
test_true = test_true.transpose(0,2,3,1)

In [32]:
print(test_pred.shape)

(12, 60, 80, 1)


In [36]:
dataset_test = cmip_dataset(X_train,true_train)
print(dataset_test.GetDataShape())

{'sst input': (166, 3, 5, 60, 80), 'sst target': (166, 1, 60, 80)}


In [37]:
t)

RuntimeError: The size of tensor a (12) must match the size of tensor b (166) at non-singleton dimension 0

In [30]:
import xarray as xr
def add(vtype,depth,data):
    
    ds=xr.open_dataset('/media/aita-ocean/data/WILL/1103Argo/mutichannel/data/BOA_Argo_annual.nc',decode_times=False)
    annual = ds[vtype].data[0,depth,49:109,159:239]
    for i in range(0,data.shape[0]):
        data[i,:,:,0] = data[i,:,:,0]+ annual
    return data

In [31]:
test_pred = add('temp', 1, test_pred)
test_true = add('temp', 1, test_true)#改

In [32]:
np.save("/media/aita-ocean/data/XGX_NEW/argo/cf/depth1_tiao1/data/test_pred.npy",test_pred)
np.save("/media/aita-ocean/data/XGX_NEW/argo/cf/depth1_tiao1/data/test_true.npy",test_true)

In [33]:
from sklearn.metrics import mean_absolute_error
def loss(vtype,depth,test_pred,test_true):
    test_preds = np.array(test_pred,copy=True)
    test_trues = np.array(test_true,copy=True)
    
    test_preds = np.squeeze(test_preds)
    test_trues = np.squeeze(test_trues)

    test_preds[np.isnan(test_preds)] = 0
    test_trues[np.isnan(test_trues)] = 0
    mask = np.load('/media/aita-ocean/data/WILL/1103Argo/mutichannel/data/'+vtype+'_'+str(depth)+'_ano.npy')
#     mask = np.squeeze(mask)
    mask = mask[0]

    total = mask.shape[0]* mask.shape[1]
    total_nan = len(mask[np.isnan(mask)])
    total_real = total - total_nan
#     print('Total NaN:',total_nan)#统计数据中的nan值
#     print('Total Real:',total_real)#统计数据中的nan值
#     #nan：0 values ：1
    mask[~np.isnan(mask)] = 1
    mask[np.isnan(mask)] = 0
    rmse = []
    rmse_temp = []
    nrmse = []
    nrmse_temp = []
    mae = []
    mae_temp = []
    for i in range(0,test_preds.shape[0]):

        final_temp = mask * test_preds[i]
        test_temp = mask * test_trues[i]
        # np.sum((y_actual - y_predicted) ** 2)
        sse = np.sum((test_temp - final_temp) ** 2)
        mse_temp = sse/total_real
        rmse_temp = np.sqrt(mse_temp)
        nrmse_temp = rmse_temp/np.mean(test_temp)
        rmse.append(rmse_temp)
        nrmse.append(nrmse_temp)
        mae_temp = mean_absolute_error(test_temp,final_temp)*total/total_real

        mae.append(mae_temp)
#     print('NAN:',len(test_pred[np.isnan(test_pred)]))
#     print('TEST NANMIN',np.nanmin(test_pred))
#     print('TEST MIN',test_pred.min())
    # print(str(depth)+'层')
    RMSE = np.sum(rmse)/len(rmse)
    MAE = np.sum(mae)/len(mae)
    NRMSE= np.sum(nrmse)/len(nrmse)
    # NRMSE = nrmse
    print(str(depth)+'层:'+'NRMSE RESULT:\n',NRMSE)
    print(str(depth)+'层:'+'RMSE RESULT:\n',RMSE)
    print(str(depth)+'层:'+'MAE RESULT:\n',MAE)

#     print('MAE RESULT:\n',MAE)

    return NRMSE

In [34]:
nrmse = loss('temp', 1, test_pred, test_true) # yao gai

1层:NRMSE RESULT:
 0.008619476484036251
1层:RMSE RESULT:
 0.22833536713995176
1层:MAE RESULT:
 0.15476821829347653


In [31]:
def corr(vtype,depth,test_pred,test_true):
    test_preds = np.array(test_pred,copy=True)
    test_trues = np.array(test_true,copy=True)
    
    test_preds = np.squeeze(test_preds)
    test_trues = np.squeeze(test_trues)

    test_preds[np.isnan(test_preds)] = 0
    test_trues[np.isnan(test_trues)] = 0
    mask = np.load('/media/aita-ocean/data/WILL/1103Argo/mutichannel/data/'+vtype+'_'+str(depth)+'_ano.npy')
#     mask = np.squeeze(mask)
    mask = mask[0]

    total = mask.shape[0]* mask.shape[1]
    total_nan = len(mask[np.isnan(mask)])
    total_real = total - total_nan
#     print('Total NaN:',total_nan)#统计数据中的nan值
#     print('Total Real:',total_real)#统计数据中的nan值
#     #nan：0 values ：1
    mask[~np.isnan(mask)] = 1
    mask[np.isnan(mask)] = 0
    CORR = []
    corr = []
    corr_temp = []
    for i in range(0,test_preds.shape[0]):

        final_temp = mask * test_preds[i]
        final_temp_f = final_temp.flatten()
        test_temp = mask * test_trues[i]
        test_temp_f = test_temp.flatten()
        corr_temp = np.corrcoef(final_temp_f,test_temp_f)[0,-1]
        # print(corr_temp)
        corr.append(corr_temp)
#     print('NAN:',len(test_pred[np.isnan(test_pred)]))
#     print('TEST NANMIN',np.nanmin(test_pred))
#     print('TEST MIN',test_pred.min())
    # print(str(depth)+'层')
    CORR = np.sum(corr)/len(corr)
    # CORR = corr
    print(str(depth)+'层:'+'CORR RESULT:\n',CORR)

#     print('MAE RESULT:\n',MAE)

    return CORR



In [32]:
cor = corr('temp', 1, test_pred, test_true)

1层:CORR RESULT:
 0.9971942156181228


: 